In [26]:
import xml.etree.ElementTree as ET
import pandas as pd

In [70]:
# Process the .gpx xml file

tree = ET.parse('GDMBR.gpx')
root = tree.getroot()

myList = []

for child in root[1][1]:
    
    lat = child.attrib['lat']
    lon = child.attrib['lon']
    ele = child[0].text
    
    myList.append([lat,lon,ele])
    
df = pd.DataFrame(myList, columns=['lat', 'lon', 'ele'])   

In [71]:
df

,lat,lon,ele
0,51.161267,-115.56016,1408.1660159
1,51.160931,-115.559829,1406.765991
2,51.160872,-115.559809,1406.886963
3,51.160814,-115.559683,1406.536011
4,51.160696,-115.559269,1405.83606
...,...,...,...
18046,31.353338,-108.53187,1423.0
18047,31.350737,-108.531126,1422.0
18048,31.335328,-108.531163,1418.0
18049,31.33343,-108.530234,1419.0
